In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
import pandas as pd
df = pd.read_csv("lead.csv")
df.sample(3)

,First Name,Last Name,Email,Company,Job Title,Industry,Company Size
2,Michael,Scott,m.scott@dundermifflin.com,Dunder Mifflin,Regional Manager,Paper & Distribution,10-50
10,Bruce,Wayne,bruce@waynetech.com,Wayne Enterprises,Chairman,Conglomerate,10000+
19,Harvey,Specter,h.specter@pearson-hardman.com,Pearson Hardman,Senior Partner,Law,200-500


In [5]:
from typing import TypedDict, Optional

class AgentState(TypedDict):
    # --- 1. INPUT DATA ---
    lead_data: dict  # Raw data from CSV (Name, Email, Company, Job Title, etc.)
    
    # --- 2. QUALIFICATION (From Scorer Agent) ---
    priority: str          # "High", "Medium", "Low"
    priority_score: int    # 1 to 10 (Useful for sorting the final CSV)
    priority_reason: str   # The "Why" behind the score
    
    # --- 3. ENRICHMENT (From Persona Agent) ---
    persona: str           # e.g., "The Data-Driven Executive"
    persona_description: str # Brief profile of their pain points
    
    # --- 4. OUTREACH (From Drafter Agent) ---
    email_subject: str
    email_body: str        # The personalized HTML/Text body
    
    # --- 5. SIMULATION (From Response Agents) ---
    simulated_reply: Optional[str]   # What the AI thinks they would say back
    response_category: Optional[str] # "Interested", "Not Interested", "Auto-reply"
    
    # --- 6. EXECUTION STATUS ---
    status: str            # "Sent", "Failed", or "Pending"

In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq # Switch later
from prompts import LEAD_SCORER_SYSTEM_PROMPT, PERSONA_ENRICHER_SYSTEM_PROMPT
from schema import LeadScore

load_dotenv()

def get_scorer_agent():
    # Model Setup
    # llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 
    # To switch to Groq later:
    llm = ChatGroq(model="qwen/qwen3-32b", temperature=0)
    
    # Bind the structured output schema to the LLM
    structured_llm = llm.with_structured_output(LeadScore)
    
    return structured_llm

# --- THE NODE FUNCTION FOR LANGGRAPH ---


In [7]:
def lead_scorer_node(state: AgentState):
    lead_data = state["lead_data"]
    
    # Format the user message with Lead details
    user_message = f"Lead Data: {lead_data}"
    
    # Invoke the agent
    scorer_agent = get_scorer_agent()
    result = scorer_agent.invoke([
        ("system", LEAD_SCORER_SYSTEM_PROMPT),
        ("human", user_message)
    ])
    
    # Update the LangGraph State
    return {
        "priority": result.priority,
        "priority_score": result.score, # Added to state for more detail
        "priority_reason": result.reasoning
    }

## **Test the lead-scorer agent**

In [8]:
# Create a dummy row from your DF
sample_lead = df.iloc[1].to_dict() # Elon Musk

# Create initial state
test_state = {
    "lead_data": sample_lead,
    "priority": "",
    "priority_score": 0,
    "priority_reason": ""
}

# Run node
updated_state = lead_scorer_node(test_state)
print(updated_state)

{'priority': 'Low', 'priority_score': 3, 'priority_reason': 'The lead is a Jr. AI Engineer at a mid-sized AI Solutions company. Junior roles typically lack decision-making authority, and while the industry is relevant, the position does not indicate high budget control or partnership potential.'}


## **Persona Enricher Node**

In [10]:

from schema import PersonaEnrichment

def get_persona_agent():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7) # Slightly higher temperature for "Creativity"
    structured_llm = llm.with_structured_output(PersonaEnrichment)
    return structured_llm

# --- THE NODE FUNCTION ---
def persona_enricher_node(state: AgentState):
    lead_data = state["lead_data"]
    
    user_message = f"Lead Data: {lead_data}"
    
    agent = get_persona_agent()
    result = agent.invoke([
        ("system", PERSONA_ENRICHER_SYSTEM_PROMPT),
        ("human", user_message)
    ])
    
    # Update the LangGraph State
    return {
        "persona": result.persona,
        "persona_description": result.persona_description
        # Note: You can also store key_motivations if you add it to AgentState
    }

In [13]:
# 1. Reset your test state
test_state = {
    "lead_data": df.iloc[2].to_dict(), # Using the Jr. AI Engineer row
    "priority": "",
    "priority_score": 0,
    "priority_reason": "",
    "persona": "",
    "persona_description": ""
}

# 2. Run Scorer and UPDATE the dictionary (Don't overwrite it)
scorer_results = lead_scorer_node(test_state)
test_state.update(scorer_results) 

# Now test_state has BOTH lead_data AND the priority results

# 3. Now run the Enricher
persona_results = persona_enricher_node(test_state)
test_state.update(persona_results)

# 4. Check the results
print(f"Priority: {test_state['priority']}")
print(f"Persona: {test_state['persona']}")
print(f"Description: {test_state['persona_description']}")

Priority: Low
Persona: The Efficiency Champion
Description: A result-driven manager focused on optimizing team performance and resource allocation. They are constantly seeking ways to enhance productivity while minimizing costs and ensuring staff satisfaction in a competitive market.


## **Draft a ostrich Email**

In [15]:
from prompts import OUTREACH_DRAFTER_SYSTEM_PROMPT
from schema import EmailDraft

def get_drafter_agent():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.8) # Higher temperature for creative writing
    structured_llm = llm.with_structured_output(EmailDraft)
    return structured_llm

# --- THE NODE FUNCTION ---
def outreach_drafter_node(state: AgentState):
    lead_data = state["lead_data"]
    priority = state["priority"]
    persona = state["persona"]
    
    # Combine context for the agent
    context = f"""
    Lead: {lead_data['First Name']} {lead_data['Last Name']}
    Role: {lead_data['Job Title']} at {lead_data['Company']}
    Priority Level: {priority}
    Buyer Persona: {persona}
    """
    
    agent = get_drafter_agent()
    result = agent.invoke([
        ("system", OUTREACH_DRAFTER_SYSTEM_PROMPT),
        ("human", f"Draft a personalized email for this lead:\n{context}")
    ])
    
    return {
        "email_subject": result.subject,
        "email_body": result.body
    }

In [16]:
# Continue from your previous 'test_state'
drafter_results = outreach_drafter_node(test_state)
test_state.update(drafter_results)

print(f"Subject: {test_state['email_subject']}")
print("-" * 30)
print(test_state['email_body'])

Subject: Quick question about streamlining Dunder Mifflin's processes
------------------------------
Hi Michael,<br><br>As the Regional Manager at Dunder Mifflin, I know you're always on the lookout for ways to boost efficiency. That's why I wanted to share how NexusAI can help you automate tedious tasks, saving your team valuable time and reducing manual work.<br><br>Open to a 5-minute chat? I'd love to explore how we can support your goals.<br><br>Best,<br>NexusAI Growth Team


## **SMTP Sender Node**

In [17]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def sender_node(state: AgentState):
    """
    Physically sends the drafted email to MailHog via SMTP.
    """
    # 1. Get data from state
    lead_email = state["lead_data"]["Email"]
    subject = state["email_subject"]
    body = state["email_body"]
    
    # 2. Setup SMTP Config
    # If running notebook locally: use "localhost"
    # If running inside Docker later: use "mailhog"
    SMTP_HOST = "localhost" 
    SMTP_PORT = 1025
    SENDER_EMAIL = "outreach@nexusai.com"

    # 3. Create the MIME Message
    message = MIMEMultipart()
    message["From"] = SENDER_EMAIL
    message["To"] = lead_email
    message["Subject"] = subject
    
    # Use "html" type because our drafter agent uses <br> tags
    message.attach(MIMEText(body, "html"))

    # 4. Attempt to send
    try:
        with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
            server.sendmail(SENDER_EMAIL, [lead_email], message.as_string())
        
        print(f"📧 Email successfully delivered to: {lead_email}")
        return {"status": "Sent"}
    
    except Exception as e:
        print(f"❌ SMTP Error: {e}")
        return {"status": f"Failed: {str(e)}"}

## **Test the mail**

In [18]:
# Continue from your test_state (which has the subject and body from the previous cell)
sender_results = sender_node(test_state)
test_state.update(sender_results)

print(f"Final Status: {test_state['status']}")

📧 Email successfully delivered to: m.scott@dundermifflin.com
Final Status: Sent


## **Feedback Loop**

In [19]:
from prompts import RESPONSE_SIMULATOR_SYSTEM_PROMPT
from schema import SimulatedResponse

def get_simulator_agent():
    # Use a higher temperature (0.9) to get diverse types of replies
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9) 
    structured_llm = llm.with_structured_output(SimulatedResponse)
    return structured_llm

# --- THE NODE FUNCTION ---
def response_simulator_node(state: AgentState):
    lead_data = state["lead_data"]
    persona = state["persona"]
    outreach_email = state["email_body"]
    
    context = f"""
    Lead: {lead_data['First Name']} {lead_data['Last Name']}
    Persona: {persona}
    Email Received: {outreach_email}
    """
    
    agent = get_simulator_agent()
    result = agent.invoke([
        ("system", RESPONSE_SIMULATOR_SYSTEM_PROMPT),
        ("human", f"Write a reply to this email based on your persona:\n{context}")
    ])
    
    return {
        "simulated_reply": result.reply_text
    }

## **Simulate a reply of the email**

In [20]:
# Continue from your test_state
simulator_results = response_simulator_node(test_state)
test_state.update(simulator_results)

print(f"Lead Name: {test_state['lead_data']['First Name']}")
print(f"Simulated Reply: {test_state['simulated_reply']}")

Lead Name: Michael
Simulated Reply: Thanks for reaching out. I'm always looking for ways to improve efficiency—let's schedule that chat for next week.


## **Categorizer agent**

In [22]:
from prompts import RESPONSE_CATEGORIZER_SYSTEM_PROMPT
from schema import ResponseCategory

def get_categorizer_agent():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) # Zero temperature for consistent labeling
    structured_llm = llm.with_structured_output(ResponseCategory)
    return structured_llm

# --- THE NODE FUNCTION ---
def response_categorizer_node(state: AgentState):
    reply = state["simulated_reply"]
    
    agent = get_categorizer_agent()
    result = agent.invoke([
        ("system", RESPONSE_CATEGORIZER_SYSTEM_PROMPT),
        ("human", f"Categorize this lead response:\n{reply}")
    ])
    
    return {
        "response_category": result.category
    }

## **Testing**

In [23]:
# Continue from your test_state
categorizer_results = response_categorizer_node(test_state)
test_state.update(categorizer_results)

print(f"Reply: {test_state['simulated_reply']}")
print(f"FINAL CATEGORY: {test_state['response_category']}")

Reply: Thanks for reaching out. I'm always looking for ways to improve efficiency—let's schedule that chat for next week.
FINAL CATEGORY: Interested


## **Its my Agent state for one lead**

In [24]:
test_state

{'lead_data': {'First Name': 'Michael',
  'Last Name': 'Scott',
  'Email': 'm.scott@dundermifflin.com',
  'Company': 'Dunder Mifflin',
  'Job Title': 'Regional Manager',
  'Industry': 'Paper & Distribution',
  'Company Size': '10-50'},
 'priority': 'Low',
 'priority_score': 3,
 'priority_reason': 'The lead is a Regional Manager at a small Paper & Distribution company with 10-50 employees. The industry is unrelated to AI/automation, and the company size suggests limited budget potential.',
 'persona': 'The Efficiency Champion',
 'persona_description': 'A result-driven manager focused on optimizing team performance and resource allocation. They are constantly seeking ways to enhance productivity while minimizing costs and ensuring staff satisfaction in a competitive market.',
 'email_subject': "Quick question about streamlining Dunder Mifflin's processes",
 'email_body': "Hi Michael,<br><br>As the Regional Manager at Dunder Mifflin, I know you're always on the lookout for ways to boost e

## **Build the Graph**

In [26]:
from langgraph.graph import StateGraph, START, END

# Initialize the Graph
workflow = StateGraph(AgentState)

# 1. Add Nodes
workflow.add_node("scorer", lead_scorer_node)
workflow.add_node("enricher", persona_enricher_node)
workflow.add_node("drafter", outreach_drafter_node)
workflow.add_node("sender", sender_node)
workflow.add_node("simulator", response_simulator_node)
workflow.add_node("categorizer", response_categorizer_node)

# 2. Define Edges (The Flow)
workflow.add_edge(START, "scorer")
workflow.add_edge("scorer", "enricher")
workflow.add_edge("enricher", "drafter")
workflow.add_edge("drafter", "sender")
workflow.add_edge("sender", "simulator")
workflow.add_edge("simulator", "categorizer")
workflow.add_edge("categorizer", END)

# 3. Compile the Graph
nexus_app = workflow.compile()

In [27]:
from prompts import CAMPAIGN_SUMMARY_PROMPT

def generate_campaign_report(results_list):
    # Prepare the data for the LLM
    summary_data = ""
    for r in results_list:
        summary_data += f"- Lead: {r['lead_data']['Email']}, Priority: {r['priority']}, Category: {r['response_category']}\n"
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    response = llm.invoke([
        ("system", CAMPAIGN_SUMMARY_PROMPT),
        ("human", f"Review these campaign results and write a summary report:\n\n{summary_data}")
    ])
    
    return response.content

In [29]:
import pandas as pd
import os

# 1. Load Leads
df = pd.read_csv("lead.csv")
final_results = []

print(f"🚀 NexusAI: Processing {len(df)} leads...")

# 2. Process each lead through the Graph
for index, row in df.iterrows():
    lead_dict = row.to_dict()
    
    # Initial State for this lead
    initial_state = {
        "lead_data": lead_dict,
        "priority": "",
        "priority_score": 0,
        "priority_reason": "",
        "persona": "",
        "persona_description": "",
        "email_subject": "",
        "email_body": "",
        "simulated_reply": "",
        "response_category": "",
        "status": "Pending"
    }
    
    # INVOKE LANGGRAPH
    print(f"[{index+1}/{len(df)}] Processing {lead_dict['Email']}...")
    result_state = nexus_app.invoke(initial_state)
    
    # Store the result
    final_results.append(result_state)

# 3. Save Enriched CSV
# We flatten the lead_data dict back into columns for a clean CSV
enriched_df = pd.DataFrame(final_results)
# Flatten lead_data into separate columns
lead_columns = pd.json_normalize(enriched_df['lead_data'])
enriched_df = pd.concat([lead_columns, enriched_df.drop(columns=['lead_data'])], axis=1)

enriched_df.to_csv("enriched_leads.csv", index=False)
print("✅ Results saved to enriched_leads.csv")

# 4. Generate & Save Markdown Report
print("📊 Generating Campaign Summary Report...")
report_content = generate_campaign_report(final_results)

# Create reports directory if it doesn't exist
os.makedirs("reports", exist_ok=True)
with open("reports/campaign_summary.md", "w") as f:
    f.write(report_content)

print("🏁 All Tasks Complete. Check 'enriched_leads.csv' and 'reports/campaign_summary.md'")

🚀 NexusAI: Processing 20 leads...
[1/20] Processing elon@spacex.com...
📧 Email successfully delivered to: elon@spacex.com
[2/20] Processing nsuer.alamin@gmail.com...
📧 Email successfully delivered to: nsuer.alamin@gmail.com
[3/20] Processing m.scott@dundermifflin.com...
📧 Email successfully delivered to: m.scott@dundermifflin.com
[4/20] Processing s.connor@cyberdyne.io...
📧 Email successfully delivered to: s.connor@cyberdyne.io
[5/20] Processing satya@microsoft.com...
📧 Email successfully delivered to: satya@microsoft.com
[6/20] Processing arman@tech-bangla.com...
📧 Email successfully delivered to: arman@tech-bangla.com
[7/20] Processing d.schrute@dundermifflin.com...
📧 Email successfully delivered to: d.schrute@dundermifflin.com
[8/20] Processing p.parker@dailybugle.com...
📧 Email successfully delivered to: p.parker@dailybugle.com
[9/20] Processing tony@starkindustries.com...
📧 Email successfully delivered to: tony@starkindustries.com
[10/20] Processing lutfur@dhaka-solutions.net...
📧